In [1]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib import colors
import matplotlib.cm as cmx
import matplotlib.gridspec as gridspec
import numpy as np
np.set_printoptions(threshold='nan')
import pandas as pd
import seaborn as sns
import sys
if sys.version_info[0] < 3:
    import Tkinter as Tk
else:
    import tkinter as Tk
    
from tkFileDialog import askopenfilename,askdirectory,asksaveasfile
sys.path.append('python')
import clusterOutliers
import keplerml

numbapro:1: ImportWarning: The numbapro package is deprecated in favour of the accelerate package. Please update your code to use equivalent functions from accelerate.
/home/dgiles/anaconda/lib/python2.7/site-packages/numba/cuda/decorators.py:107: UserWarning: autojit is deprecated and will be removed in a future release. Use jit instead.
  warn('autojit is deprecated and will be removed in a future release. Use jit instead.')


In [ ]:
reload(clusterOutliers)

In [2]:
feats = "/home/dgiles/Documents/KeplerLCs/output/Q4_output.csv"
fitsDir = "/home/dgiles/Documents/KeplerLCs/fitsFiles/Q4fitsfiles/"
Q4 = clusterOutliers.clusterOutliers(feats,fitsDir)
import pandas as pd
sample_path = "/home/dgiles/Documents/KeplerLCs/output/Q4_Sample.csv"
Q4.dataSample = pd.read_csv(sample_path,index_col=0)
Q4.filesSample = Q4.dataSample.index

In [3]:
# returns updated dataframe with pca_x and pca_y for each file
Q4_pca = Q4.pca_fit(Q4.dataSample.iloc[:,0:60])
Q4.dataSample['pca_x']=Q4_pca.T[0]
Q4.dataSample['pca_y']=Q4_pca.T[1]

In [4]:
# creates dbscan clusters based on pca reduced data
Q4.dataSample['db_pca']=Q4.db_out(Q4.dataSample[['pca_x','pca_y']])

Clustering data...

    Epsilon is in the neighborhood of 0.941727596325, 
    with an average of 1755.0 neighbors within epsilon,
    877.5 neighbors in half circle (neighbors/2).
    
Clustering data with DBSCAN...
Tabby has not Not found to be an outlier in DBSCAN
There were 1 clusters and 1587 total outliers


In [5]:
Q4.dataSample['db_tsne']=Q4.db_out(Q4.dataSample[['tsne_x','tsne_y']])

Clustering data...

    Epsilon is in the neighborhood of 1.29673900682, 
    with an average of 391.0 neighbors within epsilon,
    195.5 neighbors in half circle (neighbors/2).
    
Clustering data with DBSCAN...
Tabby has not Not found to be an outlier in DBSCAN
There were 1 clusters and 120 total outliers


In [6]:
from sklearn import preprocessing
data = Q4.dataSample.iloc[:,0:60]
scaler = preprocessing.StandardScaler().fit(data)
scaledData = scaler.transform(data)
df_scaled  = pd.DataFrame(columns=Q4.dataSample.columns[0:60],index=Q4.dataSample.index,data=scaledData)
# Returns updated dataframe with clusters based on scaled data prior to dim reduction
Q4.dataSample['db_pre'] = Q4.db_out(df_scaled)

Clustering data...

    Epsilon is in the neighborhood of 6.97464259858, 
    with an average of 1755.0 neighbors within epsilon,
    877.5 neighbors in half circle (neighbors/2).
    
Clustering data with DBSCAN...
Tabby has not Not found to be an outlier in DBSCAN
There were 1 clusters and 304 total outliers


In [ ]:
x1 = Q4.dataSample.tsne_x
y1 = Q4.dataSample.tsne_y

x2 = Q4.dataSample.pca_x
y2 = Q4.dataSample.pca_y

CL1 = Q4.dataSample.db_tsne
CL2 = Q4.dataSample.db_pca
CL3 = Q4.dataSample.db_pre


fig =plt.figure(figsize=[10,15])
fig.add_subplot(111).set_title("Title")
plt.set_cmap('nipy_spectral')

ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)

ax1.scatter(x2,y2,c=-CL1)
ax1.set_ylabel("PCA Y")
ax1.set_title("T-SNE Cluster")
ax2.scatter(x2,y2,c=-CL2)
ax2.set_ylabel("PCA Y")
ax2.set_title("PCA Cluster")
ax3.scatter(x2,y2,c=-CL3)
ax3.set_ylabel("PCA Y")
ax3.set_title("Pre Cluster")
plt.xlabel("PCA X")

fig2 = plt.figure(figsize=[10,15])
ax1 = fig2.add_subplot(311)
ax2 = fig2.add_subplot(312)
ax3 = fig2.add_subplot(313)

ax1.scatter(x1,y1,c=-CL1)
ax1.set_ylabel("t-SNE Y")
ax1.set_title("T-SNE Cluster")
ax2.scatter(x1,y1,c=-CL2)
ax2.set_ylabel("t-SNE Y")
ax2.set_title("PCA Cluster")
ax3.scatter(x1,y1,c=-CL3)
ax3.set_ylabel("t-SNE Y")
ax3.set_title("Pre Cluster")
plt.xlabel("t-SNE X")
plt.show()

In [ ]:
Q4.plot_sample(df=Q4.dataSample,pathtofits=fitsDir,cluster_method='dbscan',reduction_method='pca')

In [ ]:
from sklearn.neighbors import NearestNeighbors
data = Q4.dataSample[['pca_x','pca_y']]
data=np.array([np.array(data.loc[i]) for i in data.index])

# distance array containing all distances
nbrs = NearestNeighbors(n_neighbors=int(np.ceil(.2*len(data))), algorithm='ball_tree').fit(data)
distances, indices = nbrs.kneighbors(data)
# Distance to 2*N/100th instead of 4th because: ... reasons
neighbors = int(np.ceil(.02*len(data)))
print neighbors
distArr = distances[:,neighbors]
#distArr = distances[:,4]
#neighbors = 4
distArr.sort()
# x-axis (not meaningful)
pts = range(len(distArr))



# The following looks for the first instance (past the mid point) where the mean of the following 50 points
# is at least 5% greater than the mean of the previous 50 points.
# Alternatively, perhaps a better method, we could consider the variance of the points and draw conclusions from that
if len(data) <= 200:
    number = 10
else:
    number = 10
cutoff = 1.1
for i in range(number,len(pts)-number):
    if np.mean(distArr[i+1:i+number])>=cutoff*np.mean(distArr[i-number:i-1]) and i>(len(pts)%2+len(pts))/2:

        dbEps = distArr[i]
        break
print i
# Estimating nneighbors by finding the number of pts. 
# that fall w/in our determined eps for each point.

count = np.zeros(len(pts))
for i in pts:    
    for dist in distances[i]:
        if dist <= dbEps:
            count[i]+=1
average = np.median(count)
sigma = np.std(count)
neighbors = int(average/2) # Divide by 2 for pts on the edges of clusters
print("""
Epsilon is in the neighborhood of %s, 
with an average of %s neighbors within epsilon,
%s neighbors in half circle (neighbors/2).
"""%(dbEps,average,neighbors))

plt.scatter(pts,distArr,alpha=.2,s=30)
plt.show()

In [7]:
Q4_pca_out = [i[:13] for i in Q4.dataSample[Q4.dataSample.db_pca==-1].index]
Q4_tsne_out = [i[:13] for i in Q4.dataSample[Q4.dataSample.db_tsne==-1].index]
Q4_pre_out = [i[:13] for i in Q4.dataSample[Q4.dataSample.db_pre==-1].index]

In [8]:
Q4_pca_vs_tsne = set(Q4_pca_out).intersection(Q4_tsne_out)
Q4_pre_vs_tsne = set(Q4_pre_out).intersection(Q4_tsne_out) 
Q4_pca_vs_pre = set(Q4_pre_out).intersection(Q4_pca_out)
Q4_comp_all = set(Q4_pca_vs_tsne).intersection(Q4_pre_out)

In [10]:
print len(Q4_pre_out)
print len(Q4_pca_out)
print len(Q4_tsne_out)
print("")

print len(Q4_pca_vs_tsne)
print len(Q4_pre_vs_tsne)
print len(Q4_pca_vs_pre)
print len(Q4_comp_all)

304
1587
120

99
107
271
94


In [16]:
# Outlier in pre, not in pca.
# Plotted on PCA axes
Q4_pre_not_pca = [i for i in Q4_pre_out if i not in Q4_pca_out]
ax_tsne = plt.subplot(211)
plt.xlabel("t-SNE X")
plt.ylabel("t-SNE Y")
plt.title("Outliers in pre-reduction, not outliers after PCA")

ax_pca = plt.subplot(212)

x = Q4.dataSample.tsne_x
y = Q4.dataSample.tsne_y
ax_tsne.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_not_pca))].tsne_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_not_pca))].tsne_y
ax_tsne.scatter(x,y,c='r',alpha=1)



x = Q4.dataSample.pca_x
y = Q4.dataSample.pca_y
ax_pca.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_not_pca))].pca_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_not_pca))].pca_y
ax_pca.scatter(x,y,c='r',alpha=1)

plt.xlabel("PCA X")
plt.ylabel("PCA Y")
plt.show()

In [20]:
# Outlier in pre, not in tsne.
# Plotted on PCA axes
Q4_pre_not_tsne = [i for i in Q4_pre_out if i not in Q4_tsne_out]
ax_tsne = plt.subplot(211)
plt.xlabel("t-SNE X")
plt.ylabel("t-SNE Y")
plt.title("Outliers in pre-reduction, not outliers after t-SNE")

ax_pca = plt.subplot(212)

x = Q4.dataSample.tsne_x
y = Q4.dataSample.tsne_y
ax_tsne.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_not_tsne))].tsne_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_not_tsne))].tsne_y
ax_tsne.scatter(x,y,c='r',alpha=1)



x = Q4.dataSample.pca_x
y = Q4.dataSample.pca_y
ax_pca.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_not_tsne))].pca_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_not_tsne))].pca_y
ax_pca.scatter(x,y,c='r',alpha=1)

plt.xlabel("PCA X")
plt.ylabel("PCA Y")
plt.show()

In [21]:
# Outlier in pca, not in tsne.
# Plotted on PCA axes
Q4_pca_not_tsne = [i for i in Q4_pca_out if i not in Q4_tsne_out]
ax_tsne = plt.subplot(211)
plt.xlabel("t-SNE X")
plt.ylabel("t-SNE Y")
plt.title("Outliers in PCA reduction, not outliers in t-SNE reduction")

ax_pca = plt.subplot(212)

x = Q4.dataSample.tsne_x
y = Q4.dataSample.tsne_y
ax_tsne.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pca_not_tsne))].tsne_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pca_not_tsne))].tsne_y
ax_tsne.scatter(x,y,c='r',alpha=1)



x = Q4.dataSample.pca_x
y = Q4.dataSample.pca_y
ax_pca.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pca_not_tsne))].pca_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pca_not_tsne))].pca_y
ax_pca.scatter(x,y,c='r',alpha=1)

plt.xlabel("PCA X")
plt.ylabel("PCA Y")
plt.show()

In [23]:
# Outlier in pca, not in tsne.
# Plotted on PCA axes
Q4_pca_not_pre = [i for i in Q4_pca_out if i not in Q4_pre_out]
ax_tsne = plt.subplot(211)
plt.xlabel("t-SNE X")
plt.ylabel("t-SNE Y")
plt.title("Outliers in PCA reduction, not outliers pre-reduction")

ax_pca = plt.subplot(212)

x = Q4.dataSample.tsne_x
y = Q4.dataSample.tsne_y
ax_tsne.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pca_not_pre))].tsne_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pca_not_pre))].tsne_y
ax_tsne.scatter(x,y,c='r',alpha=1)



x = Q4.dataSample.pca_x
y = Q4.dataSample.pca_y
ax_pca.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pca_not_pre))].pca_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pca_not_pre))].pca_y
ax_pca.scatter(x,y,c='r',alpha=1)

plt.xlabel("PCA X")
plt.ylabel("PCA Y")
plt.show()

In [25]:
# Outlier in pca, not in tsne.
# Plotted on PCA axes
Q4_tsne_not_pre = [i for i in Q4_tsne_out if i not in Q4_pre_out]
ax_tsne = plt.subplot(211)
plt.xlabel("t-SNE X")
plt.ylabel("t-SNE Y")
plt.title("Outliers in t-SNE reduction, not outliers pre-reduction")

ax_pca = plt.subplot(212)

x = Q4.dataSample.tsne_x
y = Q4.dataSample.tsne_y
ax_tsne.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_tsne_not_pre))].tsne_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_tsne_not_pre))].tsne_y
ax_tsne.scatter(x,y,c='r',alpha=1)



x = Q4.dataSample.pca_x
y = Q4.dataSample.pca_y
ax_pca.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_tsne_not_pre))].pca_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_tsne_not_pre))].pca_y
ax_pca.scatter(x,y,c='r',alpha=1)

plt.xlabel("PCA X")
plt.ylabel("PCA Y")
plt.show()

In [26]:
# Outlier in pre, not in pca.
# Plotted on PCA axes
Q4_tsne_not_pca = [i for i in Q4_tsne_out if i not in Q4_pca_out]
ax_tsne = plt.subplot(211)
plt.xlabel("t-SNE X")
plt.ylabel("t-SNE Y")
plt.title("Outliers in t-SNE, not outliers in PCA")

ax_pca = plt.subplot(212)

x = Q4.dataSample.tsne_x
y = Q4.dataSample.tsne_y
ax_tsne.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_tsne_not_pca))].tsne_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_tsne_not_pca))].tsne_y
ax_tsne.scatter(x,y,c='r',alpha=1)



x = Q4.dataSample.pca_x
y = Q4.dataSample.pca_y
ax_pca.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)

x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_tsne_not_pca))].pca_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_tsne_not_pca))].pca_y
ax_pca.scatter(x,y,c='r',alpha=1)

plt.xlabel("PCA X")
plt.ylabel("PCA Y")
plt.show()

In [ ]:
x = Q4.dataSample.pca_x
y = Q4.dataSample.pca_y
plt.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)
x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_pca))].pca_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_pca))].pca_y
plt.scatter(x,y,c='r')
plt.show()

In [ ]:
x = Q4.dataSample.tsne_x
y = Q4.dataSample.tsne_y
plt.scatter(x,y,c='gray',edgecolors='gray',alpha=.5)
x = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_pca))].tsne_x
y = Q4.dataSample[Q4.dataSample.index.str.contains('|'.join(Q4_pre_pca))].tsne_y
plt.scatter(x,y,c='r')
plt.show()